In [ ]:
import os
import sys
import glob
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from minio import Minio

In [ ]:
# Define client with service account
client = Minio('s3.stxnext.pl',
                 access_key='ACCESS_KEY',
                 secret_key='SECRET_KEY',
                 secure=True)

client.list_buckets()

### Upload

In [ ]:
def upload_local_directory_to_minio(local_path: str, bucket_name: str, minio_path: str):
    """
    Uploads folder contents from local to MinIO.
    """
    assert os.path.isdir(local_path)
    
    for i, local_file in glob.glob(local_path + '/**'):
        local_file = local_file.replace(os.sep, '/')
        if not os.path.isfile(local_file):
            upload_local_directory_to_minio(local_file, bucket_name, minio_path)
        else:
            remote_path = os.path.join(minio_path, local_file)
            remote_path = remote_path.replace(os.sep, "/")
            if i%1000 == 0:
                print(remote_path)
            client.fput_object(bucket_name, remote_path, local_file)

In [ ]:
# upload_local_directory_to_minio(
#     local_path="data/models",
#     bucket_name="ml-demo",
#     minio_path="qdrant/v0.10.3/visual-similarity-search",  # VERY IMPORTANT - LOAD DATA TO A PROPER VERSION OF QDRANT
# )

In [ ]:
# upload_local_directory_to_minio(
#     local_path="data/metric_datasets",
#     bucket_name="ml-demo",
#     minio_path="qdrant/v0.10.3/visual-similarity-search",  # VERY IMPORTANT - LOAD DATA TO A PROPER VERSION OF QDRANT
# )

In [ ]:
# client.fput_object(
#     "ml-demo", 
#     "qdrant/v0.10.3/visual-similarity-search/data/qdrant_storage.zip", 
#     "data/qdrant_storage.zip"
# )

### Download

In [ ]:
metric_category = "dogs"
obj_list = client.list_objects(
    bucket_name="ml-demo",
    prefix=f"qdrant/v0.10.3/visual-similarity-search/data/metric_datasets/{metric_category}/"
)

In [ ]:
path_list = []
for i, obj in enumerate(obj_list):
    if i < 10:
        path_list.append(obj.object_name)

In [ ]:
my_object = path_list[0]
my_object

In [ ]:
from PIL import Image
import torchvision
from metrics.consts import RESIZE_TRANSFORM
from torchvision.utils import make_grid

# Get data of an object.
imgs = []
for my_object in path_list:
    try:
        response = client.get_object("ml-demo", my_object)
        # tmp_img = RESIZE_TRANSFORM(Image.open(response))
        tmp_img = Image.open(response)
        imgs.append(tmp_img)                           
    finally:
        response.close()
        response.release_conn()

In [ ]:
grid = make_grid(imgs)
grid_img = torchvision.transforms.ToPILImage()(grid)